# Darwin Dataset
This notebook will look at using the national rail DARWIN api to download and compile a list of TOC users for each stations.

In [5]:
#Import Packages
import boto3
import os
import re
import pandas as pd
import gzip
import xmltodict

In [2]:
s3 = boto3.resource(
    service_name = 's3',
    region_name='eu-west-1',
    aws_access_key_id = 'REMOVED FOR SECURITY',
    aws_secret_access_key = 'REMOVED FOR SECURITY',
    )

In [3]:
#set the bucket

directory  = "C:/Users/Mohammed/myFolder/Output"


bucket = s3.Bucket('darwin.xmltimetable')
#print the objects within the bucket
for obj in bucket.objects.all():
    if re.search("_v8.xml.gz", obj.key):
        temp_string = str(obj.key).split("/")[1]
        #print(temp_string)
        s3.Bucket('darwin.xmltimetable').download_file(obj.key, Filename = directory + "/" + temp_string)
        print(obj.key)

PPTimetable/20220523023948_v8.xml.gz
PPTimetable/20220524023843_v8.xml.gz
PPTimetable/20220525023831_v8.xml.gz
PPTimetable/20220526023801_v8.xml.gz
PPTimetable/20220527023502_v8.xml.gz
PPTimetable/20220528022757_v8.xml.gz
PPTimetable/20220529022938_v8.xml.gz
PPTimetable/20220530023632_v8.xml.gz


In [6]:
temp_array = []
filepath = "C:/Users/Mohammed/myFolder/Output/20220522022909_v8.xml.gz"

def extract_and_append(filepath):
    with gzip.open(filepath, 'rb') as f:
        file_content = f.read()
    
    #open filecontent xml
    d = xmltodict.parse(file_content)
    
    #read xml into a dataframe.
    #read only the Journey Node from the XML document
    df = pd.DataFrame.from_dict(d['PportTimetable']['Journey'])
    
    #iterate through the dataframe relevant columns and extract the values into a temp array
    cols = ["OR", "IP", "PP", "DT"]
    status = ["User", "User", "Passing Point", "User"]

    for item in zip(cols, status):
        for index, row in df.iterrows():
            #print(row["@rid"])
            TOC = row["@toc"]
            #print(row["IP"])
            if type(row[item[0]]) == list:
                for i in range(len(row[item[0]])):
                    pp_tiploc = row[item[0]][i]["@tpl"]
                    temp_array.append([TOC, item[1], pp_tiploc])

                    
                    
print(len(temp_array))
        

0


In [12]:
#iterate through a directory and call the function to append_temparray

import glob
#columns to iterate through

for filename in glob.iglob(f'{directory}/*.gz'):
    print(filename)
    extract_and_append(filename)



print(len(temp_array))    

C:/Users/Mohammed/myFolder/Output\20220522022909_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220523023948_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220524023843_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220525023831_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220526023801_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220527023502_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220528022757_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220529022938_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220530023632_v8.xml.gz
7369127


In [86]:
#convert the temp_array into a dataframe
station_tocs_original = pd.DataFrame(temp_array, columns = ["TOC","User/Pass", "TIPLOC"])

#Reduce the list by removing duplicate TOCs from Stations
station_tocs_reduced = station_tocs_original.groupby(["TOC", "TIPLOC","User/Pass"])["TOC"].count().reset_index(name="count")

station_tocs_reduced

station_tocs_reduced.to_csv(directory + "/" + "station_tocs_timetable_output.csv")
station_tocs_reduced

,TOC,TIPLOC,User/Pass,count
0,AW,ABCWM,Passing Point,924
1,AW,ABDVY,User,330
2,AW,ABER,User,1613
3,AW,ABGLELE,Passing Point,400
4,AW,ABGLELE,User,873
...,...,...,...,...
10395,ZZ,WYLYAHB,Passing Point,28
10396,ZZ,YATE,Passing Point,6
10397,ZZ,YORK,Passing Point,2
10398,ZZ,YORK,User,2


## TOC Station Probabilities
This section will look to see the probability of a TOC pulling into a station based on the number of times it has attended/departed from a particular station. 

In [123]:
#Group by Station

#Look at users
new_df_user = new_df[new_df["User/Passing Point"]=="User"]
station_probabilities = pd.DataFrame(new_df_user.groupby(["TIPLOC","TOC"]).count()).reset_index()
station_total = pd.DataFrame(new_df_user.groupby(["TIPLOC"]).count())

#station_probabilities
station_new = station_probabilities.merge(station_total, how='left', on='TIPLOC')
#station_new
station_new.to_csv("Test5.csv")
station_probabilities.to_csv("test4.csv")
#Look at passing point

In [124]:
station_new

,TIPLOC,TOC_x,User/Passing Point_x,TOC_y,User/Passing Point_y
0,ABDO,SR,88,88,88
1,ABDVY,AW,29,29,29
2,ABER,AW,119,119,119
3,ABGLELE,AW,70,70,70
4,ABHLJN,SR,1,1,1
...,...,...,...,...,...
3184,YOVILPM,SW,7,35,35
3185,YRDLYWD,LM,85,85,85
3186,YSTRADM,AW,101,101,101
3187,YTMNSTR,GW,28,28,28


In [91]:
temp2.to_csv("Test2.csv")

In [101]:
station_probabilities.groupby('TIPLOC')['TOC'].apply(' '.join).reset_index()

,TIPLOC,TOC
0,ABDO,SR
1,ABDVY,AW
2,ABER,AW
3,ABGLELE,AW
4,ABHLJN,SR
...,...,...
2533,YOVILPM,GW SW
2534,YRDLYWD,LM
2535,YSTRADM,AW
2536,YTMNSTR,GW


In [102]:
temp2.to_csv("Test2.csv")

In [107]:
temp2 = station_probabilities.groupby('TIPLOC', as_index=False).agg({'TOC' : '; '.join, 'TIPLOC' : 'first'}) 
temp2.to_csv("Test2.csv")

In [108]:
temp2

,TOC,TIPLOC
0,SR,ABDO
1,AW,ABDVY
2,AW,ABER
3,AW,ABGLELE
4,SR,ABHLJN
...,...,...
2533,GW; SW,YOVILPM
2534,LM,YRDLYWD
2535,AW,YSTRADM
2536,GW,YTMNSTR
